# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [142]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [143]:
#import dask
#dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
import numpy as np
import warnings # for muting warning messages
# mute warning messages
warnings.filterwarnings('ignore')

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [148]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
price_files = glob(PRICE_DATA+"/**/*.parquet/*.parquet")
price_df = dd.read_parquet(price_files)
price_df.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.613018,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.281464,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.782795,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.344162,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.372852,2819626,Health Care,Life Sciences Tools & Services,2000
A,2000-01-10,49.356224,49.803291,48.327969,49.311516,41.757938,2148446,Health Care,Life Sciences Tools & Services,2000
A,2000-01-11,49.311516,49.311516,47.523247,48.640915,41.190063,1855985,Health Care,Life Sciences Tools & Services,2000
A,2000-01-12,48.640915,48.640915,45.824390,47.657368,40.357174,1429874,Health Care,Life Sciences Tools & Services,2000
A,2000-01-13,48.909157,49.937412,47.210300,48.372677,40.962914,1134337,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variable Close
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [149]:
# Write your code below.
# Calculate lagged values and returns
dd_feat = (price_df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        hi_lo_range = x['High'] - x['Low']
    )
)
.assign(
    returns = lambda x: (x['Close'] / x['Close_lag_1']) - 1
))

dd_feat = dd_feat.persist()

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [150]:
# Write your code below.

# Convert the Dask data frame to a pandas data frame. 
dd_feat = dd_feat.compute()

# Add a rolling average return calculation with a window of 10 days.
grouped_data = dd_feat.groupby('ticker')

def calculate_rolling_return(group):
  min_window = min(10, len(group))
  group['rolling_return_10d'] = group['returns'].rolling(min_window).mean()
  return group

dd_feat = grouped_data.apply(calculate_rolling_return).reset_index(level=1, drop=True)

In [151]:
dd_feat.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,hi_lo_range,returns,rolling_return_10d
ticker,,,,,,,,,,,,,,
A,2012-01-03,25.636623,26.459227,25.565092,26.094421,23.452353,4156394,Health Care,Life Sciences Tools & Services,2012,NaN,0.894135,NaN,NaN
A,2012-01-04,25.858370,26.008583,25.457796,25.886980,23.265919,4651845,Health Care,Life Sciences Tools & Services,2012,26.094421,0.550787,-0.007950,NaN
A,2012-01-05,25.665236,26.695278,25.550787,26.466381,23.786655,6842651,Health Care,Life Sciences Tools & Services,2012,25.886980,1.144491,0.022382,NaN
A,2012-01-06,26.530758,26.866953,26.330473,26.752504,24.043816,4711400,Health Care,Life Sciences Tools & Services,2012,26.466381,0.536480,0.010811,NaN
A,2012-01-09,26.909870,27.482117,26.802574,27.453505,24.673838,4429563,Health Care,Life Sciences Tools & Services,2012,26.752504,0.679543,0.026203,NaN
A,2012-01-10,27.768240,28.197426,27.618027,27.968527,25.136711,4459620,Health Care,Life Sciences Tools & Services,2012,27.453505,0.579399,0.018760,NaN
A,2012-01-11,27.811159,28.111588,27.560801,27.839771,25.020988,3932574,Health Care,Life Sciences Tools & Services,2012,27.968527,0.550787,-0.004604,NaN
A,2012-01-12,27.997139,28.311874,27.439199,28.268955,25.406719,3973396,Health Care,Life Sciences Tools & Services,2012,27.839771,0.872675,0.015416,NaN
A,2012-01-13,27.939915,28.047211,27.539343,27.939915,25.110994,5058383,Health Care,Life Sciences Tools & Services,2012,28.268955,0.507868,-0.011640,NaN


```Please comment:```

#### + Was it necessary to convert to pandas to calculate the moving average return?
````
No, it was not necessary. Dask DataFrames support many pandas operations, including calculating moving averages, directly on Dask objects. This means you can compute moving averages efficiently using Dask without needing to convert your data to a pandas DataFrame first.
````
#### + Would it have been better to do it in Dask? 
```
Yes, it would have been better to calculate the moving average return in Dask, especially for handling large datasets such as stock market data.
```
#### Why?
```
1. Dask support many pandas operations, including computing moving averages
2. Dask leverages parallel computing, optimizing performance which allows to handle datasets that are larger-than-memory calculating moving averages efficiently
```

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.